## Face Detection with Multiple Cameras

In [1]:
import pandas as pd
import cv2 
import os
import time
from datetime import datetime
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
#os.chdir('C:/Users/DELL/Desktop/Praxis/Term3/Capstone/VideoDetect/Src/')

In [ ]:
cap = cv2.VideoCapture(0)
cap1 = cv2.VideoCapture(1)
while 1:
    
    rval, frame = cap.read()
    rval1, frame1 = cap1.read()
    cv2.imshow('img',frame)
    cv2.imshow('img1',frame1)
    key = cv2.waitKey(20)
    if key == 27:  # exit on ESC
        break
        
cv2.destroyAllWindows()
#cv2.destroyWindow('img')
#cv2.destroyWindow('img1')

In [2]:
import cv2
import threading

class camThread(threading.Thread):
    def __init__(self, previewName, camID, secGap, saveFolderPath):
        threading.Thread.__init__(self)
        self.previewName = previewName
        self.camID = camID
        self.secGap = secGap
        self.saveFolderPath = saveFolderPath
    def run(self):
        print("Starting " + self.previewName)
        camProcess(self.previewName, self.camID, self.secGap, self.saveFolderPath)
        
def camProcess(previewName, camID, secGap, saveFolderPath):
    cv2.namedWindow(previewName)
    cap = cv2.VideoCapture(camID)
    if cap.isOpened():  # try to get the first frame
        ret, img = cap.read()
    else:
        ret = False

    #initialise arrays and load xmls
    Left = []
    Right = []
    Timestamp = {}
    count=0
    clist=[]
    orient=[]
    seconds=[]
    prof_cascade = cv2.CascadeClassifier('C:\\Users\\DELL\\Desktop\\Praxis\\Term3\\Capstone\\VideoDetect\\myhaar_p1069n2654s20.xml')
    prof_cascade_new = cv2.CascadeClassifier('C:\\Users\\DELL\\Desktop\\Praxis\\Term3\\Capstone\\VideoDetect\\haarcascade_profileface.xml')

    # video recorder
    w = cap.get(cv2.CAP_PROP_FRAME_WIDTH);
    h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT);
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
    newpath = saveFolderPath+"Cam"+str(camID)+"\\"    # Create folders as per cam1, cam2 etc.
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    
    fileName = "Cam"+str(camID)+"_"+'1'+".mp4"   #File name in the format cam1_datetime.mpv4
    video_writer = cv2.VideoWriter(newpath+fileName, fourcc, 7.8, (int(w), int(h)))
    
    start=time.time()
    
    while 1:
        ret, img = cap.read() 
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        proffaces = prof_cascade.detectMultiScale(gray, 1.3, 5)
        
        for (x,y,w,h) in proffaces:
            
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) 
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            Right.append(datetime.now())
            orient.append("Turned Left")
            seconds.append(round((time.time()-start),2))

        proffaces_new = prof_cascade_new.detectMultiScale(gray, 1.3, 5)
        
        for (x,y,w,h) in proffaces_new:

            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) 
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            Right.append(datetime.now())
            orient.append("Turned Right")
            seconds.append(round((time.time()-start),2)) 
        count+=1
        clist.append(count)
        video_writer.write(img)
        cv2.imshow(previewName,img)
        # Wait for Esc key to stop
        k = cv2.waitKey(20) & 0xff
        if k == 27:
            break
        
    dict_stamp={'Timestamp':Right,'Orientation':orient,'Seconds':seconds}
    Stamp=pd.DataFrame(dict_stamp)
    cap.release()
    video_writer.release()
    print("Video time",time.time()-start)
    #Stamp.to_excel(newpath+"TimeStamp.xls")
    cv2.destroyAllWindows()
    calcDuration(Stamp,secGap,newpath,seconds,fileName,previewName)
    #cv2.destroyWindow(previewName)
    
    

    
    
def calcDuration(stamp,secGap,newpath,seconds,fileName,previewName):   
    sec=stamp.Seconds
    start,end=0,0
    emp_lst=[]
    dur_dict={}  #stores index against tuple of duration of farme and lenght of Frames list
    ori=stamp.Orientation

    for i in range(1,len(sec)):
        if sec[i]-sec[i-1] < secGap and ori[i-1]==ori[i]:  #time gap of 2 seconds
            emp_lst.append(sec[i-1])
            continue
        else:
            emp_lst.append(sec[i-1])

            if len(emp_lst)>3:   #4 frames per loop, minimun video size is 3 seconds,thus 3*4
                start=emp_lst[0]
                end=emp_lst[len(emp_lst)-1]
                duration=end-start
                dur_dict[i-1]=(duration,len(emp_lst)-1)
        emp_lst=[]

    elist=[]
    for i in range(len(seconds)):
        elist.append('')
    val=list(dur_dict.values())
    itr=list(dur_dict.keys())
    count=0
    for i in itr:
        x=val[count][1]
        elist[int(i)-int(x)]=val[count][0]
        elist[i]=-1
        count+=1
    df2 = stamp.copy()
    df2["Duration"]=elist
    df3 =  df2.copy()
    df3 = df3.drop('Seconds', 1)
    df3 = df3.drop(df3[(df3.Duration == "") | (df3.Duration == -1)].index)
    df3 = df3[['Timestamp','Orientation','Duration']]
    fname = previewName+".xls"
    df3.to_excel(newpath+fname)
    
    cropVideos(df2,newpath,fileName)
    
    
    
    

def cropVideos(stamp,newpath,fileName):
    from_dur=[stamp.Duration][0]
    from_sec=[stamp.Seconds][0]
    in_link = newpath+fileName
    count=0
    out_link = newpath
    for i in range(0,len(from_dur)):
        try:
            x=from_dur[i]
            if int(x)>-1:
                count+=1
                start=float(from_sec[i])
                end=float(from_sec[i]+ from_dur[i])
                ffmpeg_extract_subclip(in_link, start, end,out_link+'clip'+str(count)+'.mp4')
        except ValueError:
            pass

In [4]:
# Create two threads as follows
thread1 = camThread('Cam1', 1, 2, "C:\\Users\\DELL\\Desktop\\Praxis\\Term3\\Capstone\\VideoDetect\\Video Cropping\\final\\")
thread2 = camThread('Cam0', 0, 2, "C:\\Users\\DELL\\Desktop\\Praxis\\Term3\\Capstone\\VideoDetect\\Video Cropping\\final\\")
thread1.start()
thread2.start() 

Starting Cam1
Starting Cam0
Video time 29.73915457725525

[MoviePy] Running:
>>> C:\Users\DELL\AppData\Local\imageio\ffmpeg\ffmpeg-win32-v3.2.4.exe -y -i C:\Users\DELL\Desktop\Praxis\Term3\Capstone\VideoDetect\Video Cropping\final\Cam0\Cam0_1.mp4 -ss 7.85 -t 1.97 -vcodec copy -acodec copy C:\Users\DELL\Desktop\Praxis\Term3\Capstone\VideoDetect\Video Cropping\final\Cam0\clip1.mp4
... command successful.

[MoviePy] Running:
>>> C:\Users\DELL\AppData\Local\imageio\ffmpeg\ffmpeg-win32-v3.2.4.exe -y -i C:\Users\DELL\Desktop\Praxis\Term3\Capstone\VideoDetect\Video Cropping\final\Cam0\Cam0_1.mp4 -ss 13.21 -t 6.53 -vcodec copy -acodec copy C:\Users\DELL\Desktop\Praxis\Term3\Capstone\VideoDetect\Video Cropping\final\Cam0\clip2.mp4
... command successful.

[MoviePy] Running:
>>> C:\Users\DELL\AppData\Local\imageio\ffmpeg\ffmpeg-win32-v3.2.4.exe -y -i C:\Users\DELL\Desktop\Praxis\Term3\Capstone\VideoDetect\Video Cropping\final\Cam0\Cam0_1.mp4 -ss 20.01 -t 6.05 -vcodec copy -acodec copy C:\Users\D